<a href="https://colab.research.google.com/github/iamdsc/automatic-helmet-detection/blob/master/Helmet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training Yolo v3 Darknet model for Helmet Detection

In [0]:
# Downloading pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2019-04-07 07:20:09--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.5MB/s    in 11s     

2019-04-07 07:20:21 (20.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
# Reading darknet config and weights and creating Keras model with TF backend
import configparser
import io
import os
from collections import defaultdict

import numpy as np
from keras import backend as K
from keras.layers import (Conv2D, Input, ZeroPadding2D, Add,
                          UpSampling2D, MaxPooling2D, Concatenate)
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model as plot

Using TensorFlow backend.


In [0]:
from google.colab import files
# upload config file
files.upload()

Saving yolov3.cfg to yolov3.cfg


{'yolov3.cfg': b'[net]\r\n# Testing\r\nbatch=1\r\nsubdivisions=1\r\n# Training\r\n# batch=64\r\n# subdivisions=16\r\nwidth=416\r\nheight=416\r\nchannels=3\r\nmomentum=0.9\r\ndecay=0.0005\r\nangle=0\r\nsaturation = 1.5\r\nexposure = 1.5\r\nhue=.1\r\n\r\nlearning_rate=0.001\r\nburn_in=1000\r\nmax_batches = 500200\r\npolicy=steps\r\nsteps=400000,450000\r\nscales=.1,.1\r\n\r\n[convolutional]\r\nbatch_normalize=1\r\nfilters=32\r\nsize=3\r\nstride=1\r\npad=1\r\nactivation=leaky\r\n\r\n# Downsample\r\n\r\n[convolutional]\r\nbatch_normalize=1\r\nfilters=64\r\nsize=3\r\nstride=2\r\npad=1\r\nactivation=leaky\r\n\r\n[convolutional]\r\nbatch_normalize=1\r\nfilters=32\r\nsize=1\r\nstride=1\r\npad=1\r\nactivation=leaky\r\n\r\n[convolutional]\r\nbatch_normalize=1\r\nfilters=64\r\nsize=3\r\nstride=1\r\npad=1\r\nactivation=leaky\r\n\r\n[shortcut]\r\nfrom=-3\r\nactivation=linear\r\n\r\n# Downsample\r\n\r\n[convolutional]\r\nbatch_normalize=1\r\nfilters=128\r\nsize=3\r\nstride=2\r\npad=1\r\nactivation=le

In [0]:
def unique_config_sections(config_file):
    """Convert all config sections to have unique names.
       Adds unique suffixes to config sections for compability with configparser.
    """
    section_counters = defaultdict(int)
    output_stream = io.StringIO()
    with open(config_file) as fin:
        for line in fin:
            if line.startswith('['):
                section = line.strip().strip('[]')
                _section = section + '_' + str(section_counters[section])
                section_counters[section] += 1
                line = line.replace(section, _section)
            output_stream.write(line)
    output_stream.seek(0)
    return output_stream

In [0]:
!mkdir model_data

In [0]:
def _main():
    config_path = os.path.expanduser('yolov3.cfg')
    weights_path = os.path.expanduser('yolov3.weights')
    assert config_path.endswith('.cfg'), '{} is not a .cfg file'.format(
        config_path)
    assert weights_path.endswith(
        '.weights'), '{} is not a .weights file'.format(weights_path)

    output_path = os.path.expanduser('model_data/yolo.h5')
    assert output_path.endswith(
        '.h5'), 'output path {} is not a .h5 file'.format(output_path)
    output_root = os.path.splitext(output_path)[0]

    # Load weights and config.
    print('Loading weights.')
    weights_file = open(weights_path, 'rb')
    major, minor, revision = np.ndarray(
        shape=(3, ), dtype='int32', buffer=weights_file.read(12))
    if (major*10+minor)>=2 and major<1000 and minor<1000:
        seen = np.ndarray(shape=(1,), dtype='int64', buffer=weights_file.read(8))
    else:
        seen = np.ndarray(shape=(1,), dtype='int32', buffer=weights_file.read(4))
    print('Weights Header: ', major, minor, revision, seen)

    print('Parsing Darknet config.')
    unique_config_file = unique_config_sections(config_path)
    cfg_parser = configparser.ConfigParser()
    cfg_parser.read_file(unique_config_file)

    print('Creating Keras model.')
    input_layer = Input(shape=(None, None, 3))
    prev_layer = input_layer
    all_layers = []

    weight_decay = float(cfg_parser['net_0']['decay']
                         ) if 'net_0' in cfg_parser.sections() else 5e-4
    count = 0
    out_index = []
    for section in cfg_parser.sections():
        print('Parsing section {}'.format(section))
        if section.startswith('convolutional'):
            filters = int(cfg_parser[section]['filters'])
            size = int(cfg_parser[section]['size'])
            stride = int(cfg_parser[section]['stride'])
            pad = int(cfg_parser[section]['pad'])
            activation = cfg_parser[section]['activation']
            batch_normalize = 'batch_normalize' in cfg_parser[section]

            padding = 'same' if pad == 1 and stride == 1 else 'valid'

            # Setting weights.
            # Darknet serializes convolutional weights as:
            # [bias/beta, [gamma, mean, variance], conv_weights]
            prev_layer_shape = K.int_shape(prev_layer)

            weights_shape = (size, size, prev_layer_shape[-1], filters)
            darknet_w_shape = (filters, weights_shape[2], size, size)
            weights_size = np.product(weights_shape)

            print('conv2d', 'bn'
                  if batch_normalize else '  ', activation, weights_shape)

            conv_bias = np.ndarray(
                shape=(filters, ),
                dtype='float32',
                buffer=weights_file.read(filters * 4))
            count += filters

            if batch_normalize:
                bn_weights = np.ndarray(
                    shape=(3, filters),
                    dtype='float32',
                    buffer=weights_file.read(filters * 12))
                count += 3 * filters

                bn_weight_list = [
                    bn_weights[0],  # scale gamma
                    conv_bias,  # shift beta
                    bn_weights[1],  # running mean
                    bn_weights[2]  # running var
                ]

            conv_weights = np.ndarray(
                shape=darknet_w_shape,
                dtype='float32',
                buffer=weights_file.read(weights_size * 4))
            count += weights_size

            # DarkNet conv_weights are serialized Caffe-style:
            # (out_dim, in_dim, height, width)
            # We would like to set these to Tensorflow order:
            # (height, width, in_dim, out_dim)
            conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
            conv_weights = [conv_weights] if batch_normalize else [
                conv_weights, conv_bias
            ]

            # Handle activation.
            act_fn = None
            if activation == 'leaky':
                pass  # Add advanced activation later.
            elif activation != 'linear':
                raise ValueError(
                    'Unknown activation function `{}` in section {}'.format(
                        activation, section))

            # Create Conv2D layer
            if stride>1:
                # Darknet uses left and top padding instead of 'same' mode
                prev_layer = ZeroPadding2D(((1,0),(1,0)))(prev_layer)
            conv_layer = (Conv2D(
                filters, (size, size),
                strides=(stride, stride),
                kernel_regularizer=l2(weight_decay),
                use_bias=not batch_normalize,
                weights=conv_weights,
                activation=act_fn,
                padding=padding))(prev_layer)

            if batch_normalize:
                conv_layer = (BatchNormalization(
                    weights=bn_weight_list))(conv_layer)
            prev_layer = conv_layer

            if activation == 'linear':
                all_layers.append(prev_layer)
            elif activation == 'leaky':
                act_layer = LeakyReLU(alpha=0.1)(prev_layer)
                prev_layer = act_layer
                all_layers.append(act_layer)

        elif section.startswith('route'):
            ids = [int(i) for i in cfg_parser[section]['layers'].split(',')]
            layers = [all_layers[i] for i in ids]
            if len(layers) > 1:
                print('Concatenating route layers:', layers)
                concatenate_layer = Concatenate()(layers)
                all_layers.append(concatenate_layer)
                prev_layer = concatenate_layer
            else:
                skip_layer = layers[0]  # only one layer to route
                all_layers.append(skip_layer)
                prev_layer = skip_layer

        elif section.startswith('maxpool'):
            size = int(cfg_parser[section]['size'])
            stride = int(cfg_parser[section]['stride'])
            all_layers.append(
                MaxPooling2D(
                    pool_size=(size, size),
                    strides=(stride, stride),
                    padding='same')(prev_layer))
            prev_layer = all_layers[-1]

        elif section.startswith('shortcut'):
            index = int(cfg_parser[section]['from'])
            activation = cfg_parser[section]['activation']
            assert activation == 'linear', 'Only linear activation supported.'
            all_layers.append(Add()([all_layers[index], prev_layer]))
            prev_layer = all_layers[-1]

        elif section.startswith('upsample'):
            stride = int(cfg_parser[section]['stride'])
            assert stride == 2, 'Only stride=2 supported.'
            all_layers.append(UpSampling2D(stride)(prev_layer))
            prev_layer = all_layers[-1]

        elif section.startswith('yolo'):
            out_index.append(len(all_layers)-1)
            all_layers.append(None)
            prev_layer = all_layers[-1]

        elif section.startswith('net'):
            pass

        else:
            raise ValueError(
                'Unsupported section header type: {}'.format(section))

    # Create and save model.
    if len(out_index)==0: out_index.append(len(all_layers)-1)
    model = Model(inputs=input_layer, outputs=[all_layers[i] for i in out_index])
    print(model.summary())
    model.save('{}'.format(output_path))
    print('Saved Keras model to {}'.format(output_path))

    # Check to see if all weights have been read.
    remaining_weights = len(weights_file.read()) / 4
    weights_file.close()
    print('Read {} of {} from Darknet weights.'.format(count, count +
                                                       remaining_weights))
    if remaining_weights > 0:
        print('Warning: {} unused weights'.format(remaining_weights))

    plot(model, to_file='{}.png'.format(output_root), show_shapes=True)
    print('Saved model plot to {}.png'.format(output_root))

_main()

Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
Instructions for updating:
Colocations handled automatically by placer.
Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_1
Parsing section convolutional_7
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_2
Parsing section convolutional_9
conv2d bn leaky (3, 3, 128, 256)
Parsing section convolutional_10
conv2d bn leaky (1, 1, 256, 128)
Parsing section convolutio

In [0]:
# mounting google drive to colab to get the training data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cp -r 'drive/My Drive/JPEGImages' 'JPEGImages'

In [0]:
!ls

drive  JPEGImages  model_data  sample_data  yolov3.cfg	yolov3.weights


In [0]:
# uploading annotations file
files.upload()

Saving annotations.txt to annotations.txt


{'annotations.txt': b'JPEGImages/00001bcc92282a38.jpg 0.0,275.212288,190.5996875,199.03999999999996,0\r\nJPEGImages/000023aa04ab09ed.jpg 0.0,134.892544,328.995636,427.96031999999997,0\r\nJPEGImages/000134de87dd044b.jpg 0.0,65.801216,32.2290625,90.24,0 0.0,61.4784,170.31,108.16000000000003,0 0.0,423.624704,452.2740625,80.0,0\r\nJPEGImages/000234a17c124f75.jpg 0.0,332.72832000000005,269.35812500000003,81.92000000000002,0\r\nJPEGImages/00032b7066444e75.jpg 0.0,119.36,306.172508,258.94297600000004,0 0.0,124.47999999999999,386.2068,106.83699200000001,0 0.0,126.4,503.04042799999996,146.67366400000003,0\r\nJPEGImages/0003d1891fd14575.jpg 0.0,233.68704000000002,133.611875,44.80000000000001,0 0.0,227.92857600000002,191.31,64.63999999999999,0 0.0,275.913728,253.99062499999997,51.19999999999999,0 0.0,278.79321600000003,468.36875000000003,39.67999999999995,0 0.0,191.68696400000002,845.12,40.27250000000002,0\r\nJPEGImages/00052b203c5dad30.jpg 0.0,650.353664,352.171875,52.48000000000002,0\r\nJPEGIma

In [0]:
!mkdir yolo3

In [0]:
%cd yolo3

/content/yolo3


In [0]:
files.upload() # added model.py file
files.upload() # added utils.py file

Saving model.py to model.py


Saving utils.py to utils.py


{'utils.py': b'"""Miscellaneous utility functions."""\r\n\r\nfrom functools import reduce\r\n\r\nfrom PIL import Image\r\nimport numpy as np\r\nfrom matplotlib.colors import rgb_to_hsv, hsv_to_rgb\r\n\r\ndef compose(*funcs):\r\n    """Compose arbitrarily many functions, evaluated left to right.\r\n\r\n    Reference: https://mathieularose.com/function-composition-in-python/\r\n    """\r\n    # return lambda x: reduce(lambda v, f: f(v), funcs, x)\r\n    if funcs:\r\n        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)\r\n    else:\r\n        raise ValueError(\'Composition of empty sequence not supported.\')\r\n\r\ndef letterbox_image(image, size):\r\n    \'\'\'resize image with unchanged aspect ratio using padding\'\'\'\r\n    iw, ih = image.size\r\n    w, h = size\r\n    scale = min(w/iw, h/ih)\r\n    nw = int(iw*scale)\r\n    nh = int(ih*scale)\r\n\r\n    image = image.resize((nw,nh), Image.BICUBIC)\r\n    new_image = Image.new(\'RGB\', size, (128,128,128))\r\n   

In [0]:
!touch __init__.py

In [0]:
%cd ..

/content


In [0]:
!mkdir logs

In [0]:
# upload classes and anchors file
files.upload() # added classes.txt
files.upload() # added anchors.txt

Saving classes.txt to classes.txt


Saving anchors.txt to anchors.txt


{'anchors.txt': b'10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326\r\n'}

In [0]:
# Training the model on our data set
from keras.layers import Lambda
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from yolo3.model import preprocess_true_boxes, yolo_body, yolo_loss
from yolo3.utils import get_random_data

In [0]:
def _main():
    annotation_path = 'annotations.txt'
    log_dir = 'logs/000/'
    classes_path = 'classes.txt'
    anchors_path = 'anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)

    input_shape = (416,416) # multiple of 32, hw


    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/yolo.h5') # make sure you know what you freeze

    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 32
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint])
        model.save_weights(log_dir + 'trained_weights_stage_1.h5')

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
#     if True:
#         for i in range(len(model.layers)):
#             model.layers[i].trainable = True
#         model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
#         print('Unfreeze all of the layers.')

#         batch_size = 32 # note that more GPU memory is required after unfreezing the body
#         print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
#         model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
#             steps_per_epoch=max(1, num_train//batch_size),
#             validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
#             validation_steps=max(1, num_val//batch_size),
#             epochs=100,
#             initial_epoch=50,
#             callbacks=[logging, checkpoint, reduce_lr, early_stopping])
#         model.save_weights(log_dir + 'trained_weights_final.h5')


In [0]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model


In [0]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [0]:
# training the model
_main()

Create YOLOv3 model with 9 anchors and 1 classes.


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 18) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 18) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in shape ((1,

Load weights model_data/yolo.h5.
Freeze the first 249 layers of total 252 layers.
Train on 450 samples, val on 50 samples, with batch size 32.
Epoch 1/50
14/14 [==============================] - 69s 5s/step - loss: 10847.8363 - val_loss: 5555.7388
Epoch 2/50
14/14 [==============================] - 44s 3s/step - loss: 3133.1313 - val_loss: 1635.8392
Epoch 3/50
14/14 [==============================] - 44s 3s/step - loss: 1024.2566 - val_loss: 664.8511
Epoch 4/50
14/14 [==============================] - 20s 1s/step - loss: 515.5791 - val_loss: 409.4883
Epoch 5/50
14/14 [==============================] - 41s 3s/step - loss: 355.9752 - val_loss: 292.8072
Epoch 6/50
14/14 [==============================] - 46s 3s/step - loss: 279.0103 - val_loss: 251.6470
Epoch 7/50
14/14 [==============================] - 44s 3s/step - loss: 224.9952 - val_loss: 222.9768
Epoch 8/50
14/14 [==============================] - 46s 3s/step - loss: 195.3616 - val_loss: 176.7784
Epoch 9/50
14/14 [=================